In [ ]:
#Import drive
from google.colab import drive
#Mount Google Drive
ROOT="/content/drive"
drive.mount(ROOT, force_remount=True)

Mounted at /content/drive


In [ ]:
%pwd

'/content'

In [ ]:
%cd "/content/drive/MyDrive/dlss24/dlss24"

/content/drive/MyDrive/dlss24/dlss24


In [ ]:
!git branch

  main
* workplace


In [ ]:
!git config --global user.email "tavares.susana@outlook.com"
!git config --global user.name "susanatavares"

In [ ]:
!git pull

Already up to date.


In [ ]:
%ls

'Class 2 - Lab 28.04 - Basics .ipynb'        'Notes Class 3 - Lab 06.03 - Classes .ipynb'
'Class 3 - Lab 06.03 - Classes .ipynb'       'Notes Class 3 - Lab 06.03 - Data Analysis.ipynb'
'Class 3 - Lab 06.03 - Data Analysis.ipynb'   __pycache__/
 data/                                        README.md
 mymodule.py                                  utils.py
'Notes Class 2 - Lab 28.04 - Basics .ipynb'


For future reference: do not do this within these notes but outside in git_commands

# Week 2 - Class 2 - Data Analysis

This notebook have good example exercises to get you going on data analysis

# Coronavirus (COVID-19) Visualization & Prediction  
Coronavirus is a family of viruses that are named after their spiky crown. The novel coronavirus, also known as SARS-CoV-2, is a contagious respiratory virus that first reported in Wuhan, China. On 2/11/2020, the World Health Organization designated the name COVID-19 for the disease caused by the novel coronavirus. This notebook aims at exploring COVID-19 through data analysis and projections.

   Coronavirus Case Data is provided by <a href='https://github.com/CSSEGISandData/COVID-19'>Johns Hopkins University</a>
   <br>Learn more from the <a href='https://www.who.int/emergencies/diseases/novel-coronavirus-2019'>World Health Organization</a>
   <br>Learn more from the <a href='https://www.cdc.gov/coronavirus/2019-ncov'>Centers for Disease Control and Prevention</a>
   <br>Check out map visualizations from  <a href='https://gisanddata.maps.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6'>JHU CCSE Dashboard</a>
   <br>Source code is also on <a href='https://github.com/therealcyberlord'>my Github</a>
   
   
   ```Last update: 1/16/2023 7:41 PM ET. New Updates: Daily report data update for 1/16/23. time series data update for 1/15/23. ```

source: https://www.kaggle.com/code/therealcyberlord/coronavirus-covid-19-visualization-prediction/notebook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import random
import math
import time
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
import operator
%matplotlib inline
import warnings

plt.style.use('seaborn-poster')
warnings.filterwarnings("ignore")

numpy (arrays)

pandas (dataframes)

math - operations (absolute of a number)

time - calculate the time fo our functions, model is running and we have a process bar

machine learning package sklearn -> because it is big, add .linear etc -> subfolders inside sklearn
  inside folder x import function y, z

matplot inline -> our plots get printed wihtin notebook (usually not needed, sometimes it doesnt work, then add)

package warning -> to ignore certain warnings

seaborn -> style of plots we want to use

In [ ]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
latest_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-15-2023.csv')

In [ ]:
deaths_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7896,7896,7896,7896,7896,7896,7896,7896,7896,7896
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3598,3598,3598,3598,3598,3598,3598,3598,3598,3598
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6881,6881,6881,6881,6881,6881,6881,6881,6881,6881
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,165,165,165,165,165,165,165,165,165,165
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,1933,1933,1933,1933,1933,1933,1933,1933,1933,1933


In [ ]:
latest_data.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2023-01-16 04:21:05,33.93911,67.709953,207900,7854,NaN,NaN,Afghanistan,534.058207,3.777778
1,NaN,NaN,NaN,Albania,2023-01-16 04:21:05,41.15330,20.168300,334037,3596,NaN,NaN,Albania,11607.373688,1.076527
2,NaN,NaN,NaN,Algeria,2023-01-16 04:21:05,28.03390,1.659600,271287,6881,NaN,NaN,Algeria,618.655752,2.536428
3,NaN,NaN,NaN,Andorra,2023-01-16 04:21:05,42.50630,1.521800,47781,165,NaN,NaN,Andorra,61840.419336,0.345326
4,NaN,NaN,NaN,Angola,2023-01-16 04:21:05,-11.20270,17.873900,105095,1930,NaN,NaN,Angola,319.765542,1.836434


In [ ]:
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


In [ ]:
confirmed_df.shape

(289, 1147)

In [ ]:
def count_elements_with_characters(lst):
    count = 0
    for element in lst:
        if any('a' <= char <= 'z' for char in element):
            count += 1
    return count

"any" - list iteration. if all of this is true  -> count +

In [ ]:
def count_non_date_elements(lst):
  from datetime import datetime

  count = 0
  for element in lst:
      try:
          datetime.strptime(element, '%m/%d/%y')
      except ValueError:
          count += 1
  return count


In [ ]:
confirmed_cols = confirmed_df.columns
deaths_cols = deaths_df.columns

confirmed = confirmed_df.loc[:, confirmed_cols[4]:]
deaths = deaths_df.loc[:, deaths_cols[4]:]

In [ ]:
confirmed

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,0,0,0,0,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,0,0,0,0,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,0,0,0,0,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,0,0,0,0,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,0,0,0,0,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,0,0,0,0,0,0,0,0,0,0,...,703228,703228,703228,703228,703228,703228,703228,703228,703228,703228
285,0,0,0,0,0,0,0,0,0,0,...,535,535,535,535,535,535,535,535,535,535
286,0,0,0,0,0,0,0,0,0,0,...,11945,11945,11945,11945,11945,11945,11945,11945,11945,11945
287,0,0,0,0,0,0,0,0,0,0,...,343012,343012,343079,343079,343079,343135,343135,343135,343135,343135


In [ ]:
count_elements_with_characters(confirmed_cols)


4

In [ ]:
count_non_date_elements(confirmed)

0

In [ ]:
num_dates = len(confirmed.columns)
ck = confirmed.columns
dk = deaths.columns

world_cases = []
total_deaths = []
mortality_rate = []



In [ ]:
num_dates

1143

In [ ]:
ck

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '2/28/23', '3/1/23', '3/2/23', '3/3/23', '3/4/23', '3/5/23', '3/6/23',
       '3/7/23', '3/8/23', '3/9/23'],
      dtype='object', length=1143)

In [ ]:
dk

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '2/28/23', '3/1/23', '3/2/23', '3/3/23', '3/4/23', '3/5/23', '3/6/23',
       '3/7/23', '3/8/23', '3/9/23'],
      dtype='object', length=1143)

In [ ]:
# what is this number?
confirmed[ck[1143-1]].sum()

676570149

In [ ]:
ck

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '2/28/23', '3/1/23', '3/2/23', '3/3/23', '3/4/23', '3/5/23', '3/6/23',
       '3/7/23', '3/8/23', '3/9/23'],
      dtype='object', length=1143)

In [ ]:
ck[1143-1]

'3/9/23'

it's the last date

In [ ]:
confirmed[ck].sum()

1/22/20          557
1/23/20          657
1/24/20          944
1/25/20         1437
1/26/20         2120
             ...    
3/5/23     676024901
3/6/23     676082941
3/7/23     676213378
3/8/23     676392824
3/9/23     676570149
Length: 1143, dtype: int64

The total number of cases on the 3/9/2023

In [ ]:
num_dates = len(confirmed.columns)
ck = confirmed.columns
dk = deaths.columns

world_cases = []
total_deaths = []
mortality_rate = []

# list comprehension of this?
for i in range(num_dates): # just the dates
    confirmed_sum = confirmed[ck[i]].sum() # calculates the sum for each day
    death_sum = deaths[dk[i]].sum()

    world_cases.append(confirmed_sum)
    total_deaths.append(death_sum)

    # calculate rates
    mortality_rate.append(death_sum/confirmed_sum)

In [ ]:
world_cases = [confirmed[ck[i]].sum() for i in range(num_dates)]
total_deaths = [deaths[dk[i]].sum() for i in range(num_dates)]
mortality_rate = [death_sum[i]/confirmed_sum[i] for i in range(num_dates)]

In [ ]:
confirmed_sum

676570149

In [ ]:
range(len(world_cases))

range(0, 1143)

In [ ]:
def daily_increase(data):
    d = []
    for i in range(len(data)): # between 0 and length of our data
        if i == 0: # if we are at the first day, give value of that day (no increase)
            d.append(data[0])
        else:
            d.append(data[i]-data[i-1])
    return d # difference one day to the other

def moving_average(data, window_size): # moving average by week
    moving_average = []
    for i in range(len(data)):
        if i + window_size < len(data):
            moving_average.append(np.mean(data[i:i+window_size]))
        else:
            moving_average.append(np.mean(data[i:len(data)]))
    return moving_average

# window size
window = 7

# confirmed cases
world_daily_increase = daily_increase(world_cases)
world_confirmed_avg= moving_average(world_cases, window)
world_daily_increase_avg = moving_average(world_daily_increase, window) # average increase

# deaths
world_daily_death = daily_increase(total_deaths)
world_death_avg = moving_average(total_deaths, window)
world_daily_death_avg = moving_average(world_daily_death, window)

Want to predict world cases. We have the daily increase, the average and the moving average

In [ ]:
days_since_1_22 = np.array([i for i in range(len(ck))]).reshape(-1, 1)
# give me index of all the dates, then do reshape -> instead of long array, column array
world_cases = np.array(world_cases).reshape(-1, 1)
total_deaths = np.array(total_deaths).reshape(-1, 1)

In [ ]:
days_since_1_22

array([[   0],
       [   1],
       [   2],
       ...,
       [1140],
       [1141],
       [1142]])

In [ ]:
days_in_future = 10
future_forcast = np.array([i for i in range(len(ck)+days_in_future)]).reshape(-1, 1)
adjusted_dates = future_forcast[:-10]

In [ ]:
future_forcast

array([[   0],
       [   1],
       [   2],
       ...,
       [1150],
       [1151],
       [1152]])

forecast of what we want to check - for each of the dates, we want an array

In [ ]:
start = '1/22/2020'
start_date = datetime.datetime.strptime(start, '%m/%d/%Y')
future_forcast_dates = []
for i in range(len(future_forcast)):
    future_forcast_dates.append((start_date + datetime.timedelta(days=i)).strftime('%m/%d/%Y'))

here just defining dates as dates

In [ ]:
start_date

datetime.datetime(2020, 1, 22, 0, 0)

In [ ]:
# slightly modify the data to fit the model better (regression models cannot pick the pattern),
# we are using data from 8/1/22 and onwards for the prediction modeling
days_to_skip = 922
X_train_confirmed, \
X_test_confirmed, \
y_train_confirmed, \
y_test_confirmed = \
train_test_split(days_since_1_22[days_to_skip:], world_cases[days_to_skip:], test_size=0.07, shuffle=False)

x is date: days_since_1_22 --> 922 to end
y is all values of world cases --> 922 to end

split that so we have 0.07 test size

In [ ]:
ck[922]

'8/1/22'

In [ ]:
poly = PolynomialFeatures(degree=3)
poly_X_train_confirmed = poly.fit_transform(X_train_confirmed)
poly_X_test_confirmed = poly.fit_transform(X_test_confirmed)
poly_future_forcast = poly.fit_transform(future_forcast)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Create MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the data
poly_X_train_confirmedN = scaler.fit_transform(poly_X_train_confirmed)
poly_X_test_confirmedN = scaler.fit_transform(poly_X_test_confirmed)
y_train_confirmedN = scaler.fit_transform(y_train_confirmed)
y_test_confirmedN = scaler.fit_transform(y_test_confirmed)
poly_future_forcastN = scaler.fit_transform(poly_future_forcast)


why did we normalize the date?

In [ ]:
# polynomial regression
linear_model = LinearRegression(fit_intercept=True)
linear_model.fit(poly_X_train_confirmed, y_train_confirmed)
test_linear_pred = linear_model.predict(poly_X_test_confirmed)
linear_pred = linear_model.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_linear_pred, y_test_confirmed))
print('MSE:',mean_squared_error(test_linear_pred, y_test_confirmed))

MAE: 5510276.899736047
MSE: 31527632458016.766


In [ ]:
# polynomial regression (normalized)
linear_model = LinearRegression(fit_intercept=True)
linear_model.fit(poly_X_train_confirmedN, y_train_confirmedN)
test_linear_pred = linear_model.predict(poly_X_test_confirmedN)
linear_pred = linear_model.predict(poly_future_forcastN)
print('MAE:', mean_absolute_error(test_linear_pred, y_test_confirmedN))
print('MSE:',mean_squared_error(test_linear_pred, y_test_confirmedN))
print('MAE:', scaler.inverse_transform(test_linear_pred.reshape(-1, 1)))

MAE: 0.04671748324345974
MSE: 0.003071320107172657


ValueError: non-broadcastable output operand with shape (16,1) doesn't match the broadcast shape (16,4)

it makes no sense that 0.046 is the absolute % error - right? what does minmax transformation do? the largest number is = 1; the smallest number = 0 -> this has nothing to do with percentages, right? just normalized

In [ ]:
print(linear_model.coef_)

In [ ]:
plt.plot(y_test_confirmed)
plt.plot(test_linear_pred)
plt.legend(['Test Data', 'Polynomial Regression Predictions'])

## Q16

In [ ]:
# https://scikit-learn.org/stable/getting_started.html#fitting-and-predicting-estimator-basics
# Apply a Ramdom Forest Regressor and do the same plot


Normalizing data for a Random Forest model might not be necessary because Random Forest models are not sensitive to feature scaling due to the nature of the decision tree algorithm they are based on.

*apparently I did smth wrong lol, too simplistic*

In [ ]:
clf = RandomForestClassifier(random_state=0)
clf.fit(poly_X_train_confirmed, y_train_confirmed)
test_rf_predict = clf.predict(poly_X_test_confirmed)
rf_pred = clf.predict(poly_future_forcast)
print('MAE:', mean_absolute_error(test_rf_predict, y_test_confirmed))
print('MSE:',mean_squared_error(test_rf_predict, y_test_confirmed))

MAE: 1254608.8125
MSE: 1931766737801.3125


AttributeError: 'RandomForestClassifier' object has no attribute 'coef_'

## Q17

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris

# Load the iris dataset
iris_data = load_iris()

# Create a DataFrame
<your-answer>

# Add the target (species) column to the DataFrame
<your-answer>

# Map target numerical values to species names
<your-answer>




In [ ]:
# Display basic information
print("Basic Information:")

print("\nData Types:")


In [ ]:

print("\nSummary Statistics:")


In [ ]:

# Data Exploration
print("\nMean Values by Species:")



In [ ]:

print("\nMedian Values by Species:")


In [ ]:

print("\nMinimum Values by Species:")


In [ ]:

print("\nMaximum Values by Species:")


In [ ]:

print("\nCount of Species:")


In [ ]:
!git add "Notes Class 3 - Lab 06.03 - Data Analysis.ipynb"

In [ ]:
!git status

Refresh index: 100% (30/30), done.
On branch workplace
Your branch is up to date with 'origin/workplace'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   Notes Class 3 - Lab 06.03 - Data Analysis.ipynb

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Notes Class 2 - Lab 28.04 - Basics .ipynb
	modified:   Notes Class 3 - Lab 06.03 - Classes .ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	__pycache__/



In [ ]:
!git commit -m"Forgot to add identifier"

[workplace 9e70fe6] Forgot to add identifier
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite Notes Class 3 - Lab 06.03 - Data Analysis.ipynb (94%)


In [ ]:
!git push origin workplace

Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 9.93 KiB | 726.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/susanatavares/dlss24.git
   5f6f1ef..9e70fe6  workplace -> workplace
